In [3]:
from copy import copy
from functools import reduce

from .enums import (
    JoinType,
    UnionType,
)
from .terms import (
    ArithmeticExpression,
    EmptyCriterion,
    Field,
    Function,
    Rollup,
    Star,
    Term,
    Tuple,
    ValueWrapper,
)
from .utils import (
    JoinException,
    QueryException,
    RollupException,
    UnionException,
    alias_sql,
    builder,
    format_quotes,
    ignore_copy,
)

__author__ = "Mehak_arora"
__email__ = "mehakarora871@gmail.com"


class Selectable(object):
    def __init__(self, alias):
        self.alias = alias

    def field(self, name):
        return Field(name, table=self)

    @property
    def star(self):
        return Star(self)

    @ignore_copy
    def __getattr__(self, name):
        return self.field(name)


class AliasedQuery(Selectable):
    def __init__(self, name, query=None):
        super(AliasedQuery, self).__init__(alias=name)
        self.name = name
        self.query = query

    def get_sql(self, **kwargs):
        if self.query is None:
            return self.name
        return self.query.get_sql(**kwargs)

    def __eq__(self, other):
        return isinstance(other, AliasedQuery) \
               and self.name == other.name

    def __hash__(self):
        return hash(str(self.name))


class Schema:
    def __init__(self, name, parent=None):
        self._name = name
        self._parent = parent

    def __eq__(self, other):
        return isinstance(other, Schema) \
               and self._name == other._name \
               and self._parent == other._parent

    def __ne__(self, other):
        return not self.__eq__(other)

    def get_sql(self, quote_char=None, **kwargs):
        # FIXME escape
        schema_sql = format_quotes(self._name, quote_char)

        if self._parent is not None:
            return '{parent}.{schema}' \
                .format(parent=self._parent.get_sql(quote_char=quote_char, **kwargs),
                        schema=schema_sql)

        return schema_sql


class Table(Selectable):
    @staticmethod
    def _init_schema(schema):
        # This is a bit complicated in order to support backwards compatibility. It should probably be cleaned up for
        # the next major release. Schema is accepted as a string, list/tuple, Schema instance, or None
        if isinstance(schema, Schema):
            return schema
        if isinstance(schema, (list, tuple)):
            return reduce(lambda obj, s: Schema(s, parent=obj), schema[1:], Schema(schema[0]))
        if schema is not None:
            return Schema(schema)
        return None

    def __init__(self, name, schema=None, alias=None):
        super(Table, self).__init__(alias)
        self._table_name = name
        self._schema = self._init_schema(schema)

    def get_sql(self, quote_char=None, **kwargs):
        # FIXME escape
        table_sql = format_quotes(self._table_name, quote_char)

        if self._schema is not None:
            table_sql = '{schema}.{table}' \
                .format(schema=self._schema.get_sql(quote_char=quote_char, **kwargs),
                        table=table_sql)
        return alias_sql(table_sql, self.alias, quote_char)

    def __str__(self):
        return self.get_sql(quote_char='"')

    def __eq__(self, other):
        if not isinstance(other, Table):
            return False

        if self._table_name != other._table_name:
            return False

        if self._schema != other._schema:
            return False

        if self.alias != other.alias:
            return False

        return True

    def __repr__(self):
        if self._schema:
            return "Table('{}', schema='{}')".format(self._table_name, self._schema)
        return "Table('{}')".format(self._table_name)

    def __ne__(self, other):
        return not self.__eq__(other)

    def __hash__(self):
        return hash(str(self))


def make_tables(*names, **kwargs):
    return [Table(name, schema=kwargs.get('schema')) for name in names]


class Query(object):
    """
    Query is the primary class and entry point in pypika. It is used to build queries iteratively using the builder
    design
    pattern.
    This class is immutable.
    """

    @classmethod
    def _builder(cls):
        return QueryBuilder()

    @classmethod
    def from_(cls, table):
        """
        Query builder entry point.  Initializes query building and sets the table to select from.  When using this
        function, the query becomes a SELECT query.
        :param table:
            Type: Table or str
            An instance of a Table object or a string table name.
        :returns QueryBuilder
        """
        return cls._builder().from_(table)

    @classmethod
    def into(cls, table):
        """
        Query builder entry point.  Initializes query building and sets the table to insert into.  When using this
        function, the query becomes an INSERT query.
        :param table:
            Type: Table or str
            An instance of a Table object or a string table name.
        :returns QueryBuilder
        """
        return cls._builder().into(table)

    @classmethod
    def with_(cls, table, name):
        return cls._builder().with_(table, name)

    @classmethod
    def select(cls, *terms):
        """
        Query builder entry point.  Initializes query building without a table and selects fields.  Useful when testing
        SQL functions.
        :param terms:
            Type: list[expression]
            A list of terms to select.  These can be any type of int, float, str, bool, or Term.  They cannot be a Field
            unless the function ``Query.from_`` is called first.
        :returns QueryBuilder
        """
        return cls._builder().select(*terms)

    @classmethod
    def update(cls, table):
        """
        Query builder entry point.  Initializes query building and sets the table to update.  When using this
        function, the query becomes an UPDATE query.
        :param table:
            Type: Table or str
            An instance of a Table object or a string table name.
        :returns QueryBuilder
        """
        return cls._builder().update(table)


class _UnionQuery(Selectable, Term):
    """
    A Query class wrapper for a Union query, whether DISTINCT or ALL.
    Created via the functionds `Query.union` or `Query.union_all`, this class should not be instantiated directly.
    """

    def __init__(self, base_query, union_query, union_type, alias=None, wrapper_cls=ValueWrapper):
        super(_UnionQuery, self).__init__(alias)
        self.base_query = base_query
        self._unions = [(union_type, union_query)]
        self._orderbys = []

        self._limit = None
        self._offset = None

        self._wrapper_cls = wrapper_cls

    @builder
    def orderby(self, *fields, **kwargs):
        for field in fields:
            field = (Field(field, table=self.base_query._from[0])
                     if isinstance(field, str)
                     else self.base_query.wrap_constant(field))

            self._orderbys.append((field, kwargs.get('order')))

    @builder
    def limit(self, limit):
        self._limit = limit

    @builder
    def offset(self, offset):
        self._offset = offset

    @builder
    def union(self, other):
        self._unions.append((UnionType.distinct, other))

    @builder
    def union_all(self, other):
        self._unions.append((UnionType.all, other))

    def __add__(self, other):
        return self.union(other)

    def __mul__(self, other):
        return self.union_all(other)

    def __str__(self):
        return self.get_sql()

    def get_sql(self, with_alias=False, subquery=False, **kwargs):
        union_template = ' UNION{type} {union}'

        kwargs = {'quote_char': self.base_query.quote_char, 'dialect': self.base_query.dialect}
        base_querystring = self.base_query.get_sql(subquery=self.base_query.wrap_union_queries, **kwargs)

        querystring = base_querystring
        for union_type, union_query in self._unions:
            union_querystring = union_query.get_sql(subquery=self.base_query.wrap_union_queries, **kwargs)

            if len(self.base_query._selects) != len(union_query._selects):
                raise UnionException("Queries must have an equal number of select statements in a union."
                                     "\n\nMain Query:\n{query1}\n\nUnion Query:\n{query2}"
                                     .format(query1=base_querystring, query2=union_querystring))

            querystring += union_template.format(type=union_type.value,
                                                 union=union_querystring)

        if self._orderbys:
            querystring += self._orderby_sql(**kwargs)

        if self._limit:
            querystring += self._limit_sql()

        if self._offset:
            querystring += self._offset_sql()

        if subquery:
            querystring = '({query})'.format(query=querystring)

        if with_alias:
            return alias_sql(querystring, self.alias or self._table_name, kwargs.get('quote_char'))

        return querystring

    def _orderby_sql(self, quote_char=None, **kwargs):
        """
        Produces the ORDER BY part of the query.  This is a list of fields and possibly their directionality, ASC or
        DESC. The clauses are stored in the query under self._orderbys as a list of tuples containing the field and
        directionality (which can be None).
        If an order by field is used in the select clause, determined by a matching , then the ORDER BY clause will use
        the alias, otherwise the field will be rendered as SQL.
        """
        clauses = []
        selected_aliases = {s.alias for s in self.base_query._selects}
        for field, directionality in self._orderbys:
            term = "{quote}{alias}{quote}".format(alias=field.alias, quote=quote_char or '') \
                if field.alias and field.alias in selected_aliases \
                else field.get_sql(quote_char=quote_char, **kwargs)

            clauses.append('{term} {orient}'.format(term=term, orient=directionality.value)
                           if directionality is not None else term)

        return ' ORDER BY {orderby}'.format(orderby=','.join(clauses))

    def _offset_sql(self):
        return " OFFSET {offset}".format(offset=self._offset)

    def _limit_sql(self):
        return " LIMIT {limit}".format(limit=self._limit)


class QueryBuilder(Selectable, Term):
    """
    Query Builder is the main class in pypika which stores the state of a query and offers functions which allow the
    state to be branched immutably.
    """

    def __init__(self, quote_char='"', dialect=None, wrap_union_queries=True, wrapper_cls=ValueWrapper):
        super(QueryBuilder, self).__init__(None)

        self._from = []
        self._insert_table = None
        self._update_table = None
        self._delete_from = False

        self._with = []
        self._selects = []
        self._columns = []
        self._values = []
        self._distinct = False
        self._ignore = False

        self._wheres = None
        self._prewheres = None
        self._groupbys = []
        self._with_totals = False
        self._havings = None
        self._orderbys = []
        self._joins = []
        self._unions = []

        self._limit = None
        self._offset = None

        self._updates = []

        self._select_star = False
        self._select_star_tables = set()
        self._mysql_rollup = False
        self._select_into = False

        self._subquery_count = 0
        self._foreign_table = False

        self.quote_char = quote_char
        self.dialect = dialect
        self.wrap_union_queries = wrap_union_queries

        self._wrapper_cls = wrapper_cls

    def __copy__(self):
        newone = type(self).__new__(type(self))
        newone.__dict__.update(self.__dict__)
        newone._select_star_tables = copy(self._select_star_tables)
        newone._from = copy(self._from)
        newone._with = copy(self._with)
        newone._selects = copy(self._selects)
        newone._columns = copy(self._columns)
        newone._values = copy(self._values)
        newone._groupbys = copy(self._groupbys)
        newone._orderbys = copy(self._orderbys)
        newone._joins = copy(self._joins)
        newone._unions = copy(self._unions)
        newone._updates = copy(self._updates)
        return newone

    @builder
    def from_(self, selectable):
        """
        Adds a table to the query.  This function can only be called once and will raise an AttributeError if called a
        second time.
        :param selectable:
            Type: ``Table``, ``Query``, or ``str``
            When a ``str`` is passed, a table with the name matching the ``str`` value is used.
        :returns
            A copy of the query with the table added.
        """

        self._from.append(Table(selectable) if isinstance(selectable, str) else selectable)

        if isinstance(selectable, (QueryBuilder, _UnionQuery)) and selectable.alias is None:
            if isinstance(selectable, QueryBuilder):
                sub_query_count = selectable._subquery_count
            else:
                sub_query_count = 0

            sub_query_count = max(self._subquery_count, sub_query_count)
            selectable.alias = 'sq%d' % sub_query_count
            self._subquery_count = sub_query_count + 1

    @builder
    def with_(self, selectable, name):
        t = AliasedQuery(name, selectable)
        self._with.append(t)

    @builder
    def into(self, table):
        if self._insert_table is not None:
            raise AttributeError("'Query' object has no attribute '%s'" % 'into')

        if self._selects:
            self._select_into = True

        self._insert_table = table if isinstance(table, Table) else Table(table)

    @builder
    def select(self, *terms):
        for term in terms:
            if isinstance(term, Field):
                self._select_field(term)
            elif isinstance(term, str):
                self._select_field_str(term)
            elif isinstance(term, (Function, ArithmeticExpression)):
                self._select_other(term)
            else:
                self._select_other(self.wrap_constant(term))

    @builder
    def delete(self):
        if self._delete_from or self._selects or self._update_table:
            raise AttributeError("'Query' object has no attribute '%s'" % 'delete')

        self._delete_from = True

    @builder
    def update(self, table):
        if self._update_table is not None or self._selects or self._delete_from:
            raise AttributeError("'Query' object has no attribute '%s'" % 'update')

        self._update_table = table if isinstance(table, Table) else Table(table)

    @builder
    def columns(self, *terms):
        if self._insert_table is None:
            raise AttributeError("'Query' object has no attribute '%s'" % 'insert')

        for term in terms:
            if isinstance(term, str):
                term = Field(term, table=self._insert_table)
            self._columns.append(term)

    @builder
    def insert(self, *terms):
        if self._insert_table is None:
            raise AttributeError("'Query' object has no attribute '%s'" % 'insert')

        if not terms:
            return

        if not isinstance(terms[0], (list, tuple, set)):
            terms = [terms]

        for values in terms:
            self._values.append([value
                                 if isinstance(value, Term)
                                 else self.wrap_constant(value)
                                 for value in values])

    @builder
    def distinct(self):
        self._distinct = True

    @builder
    def ignore(self):
        self._ignore = True

    @builder
    def prewhere(self, criterion):
        if not self._validate_table(criterion):
            self._foreign_table = True

        if self._prewheres:
            self._prewheres &= criterion
        else:
            self._prewheres = criterion

    @builder
    def where(self, criterion):
        if isinstance(criterion, EmptyCriterion):
            return

        if not self._validate_table(criterion):
            self._foreign_table = True

        if self._wheres:
            self._wheres &= criterion
        else:
            self._wheres = criterion

    @builder
    def having(self, criterion):
        if self._havings:
            self._havings &= criterion
        else:
            self._havings = criterion

    @builder
    def groupby(self, *terms):
        for term in terms:
            if isinstance(term, str):
                term = Field(term, table=self._from[0])
            elif isinstance(term, int):
                term = Field(str(term), table=self._from[0]).wrap_constant(term)

            self._groupbys.append(term)

    @builder
    def with_totals(self):
        self._with_totals = True

    @builder
    def rollup(self, *terms, **kwargs):
        for_mysql = 'mysql' == kwargs.get('vendor')

        if self._mysql_rollup:
            raise AttributeError("'Query' object has no attribute '%s'" % 'rollup')

        terms = [Tuple(*term) if isinstance(term, (list, tuple, set))
                 else term
                 for term in terms]

        if for_mysql:
            # MySQL rolls up all of the dimensions always
            if not terms and not self._groupbys:
                raise RollupException('At least one group is required. Call Query.groupby(term) or pass'
                                      'as parameter to rollup.')

            self._mysql_rollup = True
            self._groupbys += terms

        elif 0 < len(self._groupbys) and isinstance(self._groupbys[-1], Rollup):
            # If a rollup was added last, then append the new terms to the previous rollup
            self._groupbys[-1].args += terms

        else:
            self._groupbys.append(Rollup(*terms))

    @builder
    def orderby(self, *fields, **kwargs):
        for field in fields:
            field = (Field(field, table=self._from[0])
                     if isinstance(field, str)
                     else self.wrap_constant(field))

            self._orderbys.append((field, kwargs.get('order')))

    @builder
    def join(self, item, how=JoinType.inner):
        if isinstance(item, Table):
            return Joiner(self, item, how, type_label='table')

        elif isinstance(item, QueryBuilder):
            return Joiner(self, item, how, type_label='subquery')

        elif isinstance(item, AliasedQuery):
            return Joiner(self, item, how, type_label='table')

        raise ValueError("Cannot join on type '%s'" % type(item))

    @builder
    def limit(self, limit):
        self._limit = limit

    @builder
    def offset(self, offset):
        self._offset = offset

    @builder
    def union(self, other):
        return _UnionQuery(self, other, UnionType.distinct, wrapper_cls=self._wrapper_cls)

    @builder
    def union_all(self, other):
        return _UnionQuery(self, other, UnionType.all, wrapper_cls=self._wrapper_cls)

    @builder
    def set(self, field, value):
        field = Field(field) if not isinstance(field, Field) else field
        self._updates.append((field, ValueWrapper(value)))

    def __add__(self, other):
        return self.union(other)

    def __mul__(self, other):
        return self.union_all(other)

    @builder
    def __getitem__(self, item):
        if not isinstance(item, slice):
            raise TypeError("Query' object is not subscriptable")
        self._offset = item.start
        self._limit = item.stop

    @staticmethod
    def _list_aliases(field_set, quote_char=None):
        return [field.alias or field.get_sql(quote_char=quote_char)
                for field in field_set]

    def _select_field_str(self, term):
        if 0 == len(self._from):
            raise QueryException('Cannot select {term}, no FROM table specified.'
                                 .format(term=term))

        if term == '*':
            self._select_star = True
            self._selects = [Star()]
            return

        self._select_field(Field(term, table=self._from[0]))

    def _select_field(self, term):
        if self._select_star:
            # Do not add select terms after a star is selected
            return

        if term.table in self._select_star_tables:
            # Do not add select terms for table after a table star is selected
            return

        if isinstance(term, Star):
            self._selects = [select
                             for select in self._selects
                             if not hasattr(select, 'table') or term.table != select.table]
            self._select_star_tables.add(term.table)

        self._selects.append(term)

    def _select_other(self, function):
        self._selects.append(function)

    def fields(self):
        # Don't return anything here. Subqueries have their own fields.
        return []

    def do_join(self, join):
        base_tables = self._from + [self._update_table] + self._with

        join.validate(base_tables, self._joins)

        if isinstance(join.item, QueryBuilder) and join.item.alias is None:
            self._tag_subquery(join.item)

        table_in_query = any(isinstance(clause, Table)
                             and join.item in base_tables
                             for clause in base_tables)
        if isinstance(join.item, Table) and join.item.alias is None and table_in_query:
            # On the odd chance that we join the same table as the FROM table and don't set an alias
            # FIXME only works once
            join.item.alias = join.item._table_name + '2'

        self._joins.append(join)

    def _validate_table(self, term):
        """
        Returns False if the term references a table not already part of the
        FROM clause or JOINS and True otherwise.
        """
        base_tables = self._from + [self._update_table]

        for field in term.fields():
            table_in_base_tables = field.table in base_tables
            table_in_joins = field.table in [join.item for join in self._joins]
            if field.table is not None \
                  and not table_in_base_tables \
                  and not table_in_joins \
                  and field.table != self._update_table:
                return False
        return True

    def _tag_subquery(self, subquery):
        subquery.alias = 'sq%d' % self._subquery_count
        self._subquery_count += 1

    def __str__(self):
        return self.get_sql(quote_char=self.quote_char, dialect=self.dialect)

    def __repr__(self):
        return self.__str__()

    def __eq__(self, other):
        if not isinstance(other, QueryBuilder):
            return False

        if not self.alias == other.alias:
            return False

        return True

    def __ne__(self, other):
        return not self.__eq__(other)

    def __hash__(self):
        return hash(self.alias) + sum(hash(clause) for clause in self._from)

    def get_sql(self, with_alias=False, subquery=False, **kwargs):
        kwargs.setdefault('quote_char', self.quote_char)

        if not (self._selects or self._insert_table or self._delete_from or self._update_table):
            return ''
        if self._insert_table and not (self._selects or self._values):
            return ''
        if self._update_table and not self._updates:
            return ''

        has_joins = bool(self._joins)
        has_multiple_from_clauses = 1 < len(self._from)
        has_subquery_from_clause = 0 < len(self._from) and isinstance(self._from[0], QueryBuilder)
        has_reference_to_foreign_table = self._foreign_table

        kwargs['with_namespace'] = any((
            has_joins,
            has_multiple_from_clauses,
            has_subquery_from_clause,
            has_reference_to_foreign_table
        ))

        if self._update_table:
            querystring = self._update_sql(**kwargs)

            if self._joins:
                querystring += " " + " ".join(join.get_sql(**kwargs)
                                              for join in self._joins)

            querystring += self._set_sql(**kwargs)

            if self._wheres:
                querystring += self._where_sql(**kwargs)

            return querystring

        if self._delete_from:
            querystring = self._delete_sql(**kwargs)

        elif not self._select_into and self._insert_table:
            querystring = self._insert_sql(**kwargs)

            if self._columns:
                querystring += self._columns_sql(**kwargs)

            if self._values:
                querystring += self._values_sql(**kwargs)
                return querystring
            else:
                querystring += ' ' + self._select_sql(**kwargs)

        else:
            if self._with:
                querystring = self._with_sql(**kwargs)
            else:
                querystring = ''

            querystring += self._select_sql(**kwargs)

            if self._insert_table:
                querystring += self._into_sql(**kwargs)

        if self._from:
            querystring += self._from_sql(**kwargs)

        if self._joins:
            querystring += " " + " ".join(join.get_sql(**kwargs)
                                          for join in self._joins)

        if self._prewheres:
            querystring += self._prewhere_sql(**kwargs)

        if self._wheres:
            querystring += self._where_sql(**kwargs)

        if self._groupbys:
            querystring += self._group_sql(**kwargs)
            if self._mysql_rollup:
                querystring += self._rollup_sql()

        if self._havings:
            querystring += self._having_sql(**kwargs)

        if self._orderbys:
            querystring += self._orderby_sql(**kwargs)

        if self._limit:
            querystring += self._limit_sql()

        if self._offset:
            querystring += self._offset_sql()

        if subquery:
            querystring = '({query})'.format(query=querystring)

        if with_alias:
            return alias_sql(querystring, self.alias, kwargs.get('quote_char'))

        return querystring

    def _with_sql(self, **kwargs):
        return 'WITH ' + ','.join(
              clause.name + ' AS (' + clause.get_sql(
                    subquery=False,
                    with_alias=False,
                    **kwargs) +
              ') '
              for clause in self._with)

    def _select_sql(self, **kwargs):
        return 'SELECT {distinct}{select}'.format(
              distinct='DISTINCT ' if self._distinct else '',
              select=','.join(term.get_sql(with_alias=True, subquery=True, **kwargs)
                              for term in self._selects),
        )

    def _insert_sql(self, **kwargs):
        return 'INSERT {ignore}INTO {table}'.format(
              table=self._insert_table.get_sql(**kwargs),
              ignore='IGNORE ' if self._ignore else ''
        )

    @staticmethod
    def _delete_sql(**kwargs):
        return 'DELETE'

    def _update_sql(self, **kwargs):
        return 'UPDATE {table}'.format(
              table=self._update_table.get_sql(**kwargs)
        )

    def _columns_sql(self, with_namespace=False, **kwargs):
        """
        SQL for Columns clause for INSERT queries
        :param with_namespace:
            Remove from kwargs, never format the column terms with namespaces since only one table can be inserted into
        """
        return ' ({columns})'.format(
              columns=','.join(term.get_sql(with_namespace=False, **kwargs)
                               for term in self._columns)
        )

    def _values_sql(self, **kwargs):
        return ' VALUES ({values})' \
            .format(values='),('
                    .join(','
                          .join(term.get_sql(with_alias=True, subquery=True, **kwargs)
                                   for term in row)
                          for row in self._values))

    def _into_sql(self, **kwargs):
        return ' INTO {table}'.format(
              table=self._insert_table.get_sql(with_alias=False, **kwargs),
        )

    def _from_sql(self, with_namespace=False, **kwargs):
        return ' FROM {selectable}'.format(selectable=','.join(
              clause.get_sql(subquery=True, with_alias=True, **kwargs)
              for clause in self._from
        ))

    def _prewhere_sql(self, quote_char=None, **kwargs):
        return ' PREWHERE {prewhere}'.format(
              prewhere=self._prewheres.get_sql(quote_char=quote_char, subquery=True, **kwargs))

    def _where_sql(self, quote_char=None, **kwargs):
        return ' WHERE {where}'.format(where=self._wheres.get_sql(quote_char=quote_char, subquery=True, **kwargs))

    def _group_sql(self, quote_char=None, groupby_alias=True, **kwargs):
        """
        Produces the GROUP BY part of the query.  This is a list of fields. The clauses are stored in the query under
        self._groupbys as a list fields.
        If an groupby field is used in the select clause,
        determined by a matching alias, and the groupby_alias is set True
        then the GROUP BY clause will use the alias,
        otherwise the entire field will be rendered as SQL.
        """
        clauses = []
        selected_aliases = {s.alias for s in self._selects}
        for field in self._groupbys:
            if groupby_alias and field.alias and field.alias in selected_aliases:
                clauses.append("{quote}{alias}{quote}".format(
                      alias=field.alias,
                      quote=quote_char or '',
                ))
            else:
                clauses.append(field.get_sql(quote_char=quote_char, **kwargs))

        sql = ' GROUP BY {groupby}'.format(groupby=','.join(clauses))
        if self._with_totals:
            return sql + ' WITH TOTALS'
        return sql

    def _orderby_sql(self, quote_char=None, orderby_alias=True, **kwargs):
        """
        Produces the ORDER BY part of the query.  This is a list of fields and possibly their directionality, ASC or
        DESC. The clauses are stored in the query under self._orderbys as a list of tuples containing the field and
        directionality (which can be None).
        If an order by field is used in the select clause,
        determined by a matching, and the orderby_alias
        is set True then the ORDER BY clause will use
        the alias, otherwise the field will be rendered as SQL.
        """
        clauses = []
        selected_aliases = {s.alias for s in self._selects}
        for field, directionality in self._orderbys:
            term = "{quote}{alias}{quote}".format(alias=field.alias, quote=quote_char or '') \
                if orderby_alias and field.alias and field.alias in selected_aliases \
                else field.get_sql(quote_char=quote_char, **kwargs)

            clauses.append('{term} {orient}'.format(term=term, orient=directionality.value)
                           if directionality is not None else term)

        return ' ORDER BY {orderby}'.format(orderby=','.join(clauses))

    def _rollup_sql(self):
        return ' WITH ROLLUP'

    def _having_sql(self, quote_char=None, **kwargs):
        return ' HAVING {having}'.format(having=self._havings.get_sql(quote_char=quote_char, **kwargs))

    def _offset_sql(self):
        return " OFFSET {offset}".format(offset=self._offset)

    def _limit_sql(self):
        return " LIMIT {limit}".format(limit=self._limit)

    def _set_sql(self, **kwargs):
        return ' SET {set}'.format(
              set=','.join(
                    '{field}={value}'.format(
                          field=field.get_sql(**kwargs),
                          value=value.get_sql(**kwargs)) for field, value in self._updates
              )
        )


class Joiner(object):
    def __init__(self, query, item, how, type_label):
        self.query = query
        self.item = item
        self.how = how
        self.type_label = type_label

    def on(self, criterion):
        if criterion is None:
            raise JoinException("Parameter 'criterion' is required for a "
                                "{type} JOIN but was not supplied.".format(type=self.type_label))

        self.query.do_join(JoinOn(self.item, self.how, criterion))
        return self.query

    def on_field(self, *fields):
        if not fields:
            raise JoinException("Parameter 'fields' is required for a "
                                "{type} JOIN but was not supplied.".format(type=self.type_label))

        criterion = None
        for field in fields:
            consituent = Field(field, table=self.query._from[0]) == Field(field, table=self.item)
            criterion = consituent if criterion is None else criterion & consituent

        self.query.do_join(JoinOn(self.item, self.how, criterion))
        return self.query

    def using(self, *fields):
        if not fields:
            raise JoinException("Parameter 'fields' is required when joining with "
                                "a using clause but was not supplied.".format(type=self.type_label))

        self.query.do_join(JoinUsing(self.item, self.how, [Field(field) for field in fields]))
        return self.query

    def cross(self):
        """Return cross join"""
        self.query.do_join(Join(self.item, JoinType.cross))

        return self.query


class Join(object):
    def __init__(self, item, how):
        self.item = item
        self.how = how

    def get_sql(self, **kwargs):
        sql = 'JOIN {table}'.format(
              table=self.item.get_sql(subquery=True, with_alias=True, **kwargs),
        )

        if self.how.value:
            return '{type} {join}'.format(join=sql, type=self.how.value)
        return sql

    def validate(self, _from, _joins):
        pass


class JoinOn(Join):
    def __init__(self, item, how, criteria):
        super(JoinOn, self).__init__(item, how)
        self.criterion = criteria

    def get_sql(self, **kwargs):
        join_sql = super(JoinOn, self).get_sql(**kwargs)
        return '{join} ON {criterion}'.format(
              join=join_sql,
              criterion=self.criterion.get_sql(**kwargs),
        )

    def validate(self, _from, _joins):
        criterion_tables = set([f.table for f in self.criterion.fields()])
        available_tables = (set(_from) | {join.item for join in _joins} | {self.item})
        missing_tables = criterion_tables - available_tables
        if missing_tables:
            raise JoinException('Invalid join criterion. One field is required from the joined item and '
                                'another from the selected table or an existing join.  Found [{tables}]'.format(
                  tables=', '.join(map(str, missing_tables))
            ))


class JoinUsing(Join):
    def __init__(self, item, how, fields):
        super(JoinUsing, self).__init__(item, how)
        self.fields = fields

    def get_sql(self, **kwargs):
        join_sql = super(JoinUsing, self).get_sql(**kwargs)
        return '{join} USING ({fields})'.format(
              join=join_sql,
              fields=','.join(str(field) for field in self.fields)
        )

    def validate(self, _from, _joins):
        pass

ModuleNotFoundError: No module named '__main__.enums'; '__main__' is not a package

In [ ]:
"""
PyPika is divided into a couple of modules, primarily the ``queries`` and ``terms`` modules.
pypika.queries
--------------
This is where the ``Query`` class can be found which is the core class in PyPika.  Also, other top level classes such
as ``Table`` can be found here.  ``Query`` is a container that holds all of the ``Term`` types together and also
serializes the builder to a string.
pypika.terms
------------
This module contains the classes which represent individual parts of queries that extend the ``Term`` base class.
pypika.functions
----------------
Wrappers for common SQL functions are stored in this package.
pypika.enums
------------
Enumerated values are kept in this package which are used as options for Queries and Terms.
pypika.utils
------------
This contains all of the utility classes such as exceptions and decorators.
"""x
# noinspection PyUnresolvedReferences
from .dialects import (
    ClickHouseQuery,
    Dialects,
    MSSQLQuery,
    MySQLQuery,
    OracleQuery,
    PostgreSQLQuery,
    RedshiftQuery,
    SQLLiteQuery,
    VerticaQuery,
)
# noinspection PyUnresolvedReferences
from .enums import (
    DatePart,
    JoinType,
    Order,
)
# noinspection PyUnresolvedReferences
from .queries import (
    AliasedQuery,
    Query,
    Schema,
    Table,
    make_tables as Tables,
)
# noinspection PyUnresolvedReferences
from .terms import (
    Array,
    Bracket,
    Case,
    Criterion,
    EmptyCriterion,
    Field,
    Interval,
    Not,
    NullValue,
    Parameter,
    Rollup,
    Tuple,
)
# noinspection PyUnresolvedReferences
from .utils import (
    CaseException,
    GroupingException,
    JoinException,
    QueryException,
    RollupException,
    UnionException,
)

__author__ = 'Timothy Heys'
__email__ = 'theys@kayak.com'
__version__ = '0.27.0'

In [ ]:
# coding: utf-8
"""
Package for SQL analytic functions wrappers
"""
from pypika.terms import (AnalyticFunction,
                          WindowFrameAnalyticFunction,
                          IgnoreNullsAnalyticFunction)

__author__ = "Timothy Heys"
__email__ = "theys@kayak.com"


class Preceding(WindowFrameAnalyticFunction.Edge):
    modifier = 'PRECEDING'


class Following(WindowFrameAnalyticFunction.Edge):
    modifier = 'FOLLOWING'


CURRENT_ROW = 'CURRENT ROW'


class Rank(AnalyticFunction):
    def __init__(self, **kwargs):
        super(Rank, self).__init__('RANK', **kwargs)


class DenseRank(AnalyticFunction):
    def __init__(self, **kwargs):
        super(DenseRank, self).__init__('DENSE_RANK', **kwargs)


class RowNumber(AnalyticFunction):
    def __init__(self, **kwargs):
        super(RowNumber, self).__init__('ROW_NUMBER', **kwargs)


class NTile(AnalyticFunction):
    def __init__(self, term, **kwargs):
        super(NTile, self).__init__('NTILE', term, **kwargs)


class FirstValue(WindowFrameAnalyticFunction, IgnoreNullsAnalyticFunction):
    def __init__(self, *terms, **kwargs):
        super(FirstValue, self).__init__('FIRST_VALUE', *terms, **kwargs)


class LastValue(WindowFrameAnalyticFunction, IgnoreNullsAnalyticFunction):
    def __init__(self, *terms, **kwargs):
        super(LastValue, self).__init__('LAST_VALUE', *terms, **kwargs)


class Median(AnalyticFunction):
    def __init__(self, term, **kwargs):
        super(Median, self).__init__('MEDIAN', term, **kwargs)


class Avg(WindowFrameAnalyticFunction):
    def __init__(self, term, **kwargs):
        super(Avg, self).__init__('AVG', term, **kwargs)


class StdDev(WindowFrameAnalyticFunction):
    def __init__(self, term, **kwargs):
        super(StdDev, self).__init__('STDDEV', term, **kwargs)


class StdDevPop(WindowFrameAnalyticFunction):
    def __init__(self, term, **kwargs):
        super(StdDevPop, self).__init__('STDDEV_POP', term, **kwargs)


class StdDevSamp(WindowFrameAnalyticFunction):
    def __init__(self, term, **kwargs):
        super(StdDevSamp, self).__init__('STDDEV_SAMP', term, **kwargs)


class Variance(WindowFrameAnalyticFunction):
    def __init__(self, term, **kwargs):
        super(Variance, self).__init__('VARIANCE', term, **kwargs)


class VarPop(WindowFrameAnalyticFunction):
    def __init__(self, term, **kwargs):
        super(VarPop, self).__init__('VAR_POP', term, **kwargs)


class VarSamp(WindowFrameAnalyticFunction):
    def __init__(self, term, **kwargs):
        super(VarSamp, self).__init__('VAR_SAMP', term, **kwargs)


class Count(WindowFrameAnalyticFunction):
    def __init__(self, term, **kwargs):
        super(Count, self).__init__('COUNT', term, **kwargs)


class Sum(WindowFrameAnalyticFunction):
    def __init__(self, term, **kwargs):
        super(Sum, self).__init__('SUM', term, **kwargs)


class Max(WindowFrameAnalyticFunction):
    def __init__(self, term, **kwargs):
        super(Max, self).__init__('MAX', term, **kwargs)


class Min(WindowFrameAnalyticFunction):
    def __init__(self, term, **kwargs):
        super(Min, self).__init__('MIN', term, **kwargs)

In [ ]:
# coding: utf-8
from aenum import Enum

__author__ = "Timothy Heys"
__email__ = "theys@kayak.com"


class Arithmetic(Enum):
    add = '+'
    sub = '-'
    mul = '*'
    div = '/'


class Comparator(Enum):
    pass


class Equality(Comparator):
    eq = '='
    ne = '<>'
    gt = '>'
    gte = '>='
    lt = '<'
    lte = '<='


class Matching(Comparator):
    not_like = ' NOT LIKE '
    like = ' LIKE '
    not_ilike = ' NOT ILIKE '
    ilike = ' ILIKE '
    regex = ' REGEX '
    bin_regex = ' REGEX BINARY '


class Boolean(Comparator):
    and_ = 'AND'
    or_ = 'OR'
    xor_ = 'XOR'
    true = 'TRUE'
    false = 'FALSE'


class Order(Enum):
    asc = 'ASC'
    desc = 'DESC'


class JoinType(Enum):
    inner = ''
    left = 'LEFT'
    right = 'RIGHT'
    outer = 'FULL OUTER'
    left_outer = 'LEFT OUTER'
    right_outer = 'RIGHT OUTER'
    full_outer = 'FULL OUTER'
    cross = 'CROSS'


class UnionType(Enum):
    distinct = ''
    all = ' ALL'


class DatePart(Enum):
    year = 'YEAR'
    quarter = 'QUARTER'
    month = 'MONTH'
    week = 'WEEK'
    day = 'DAY'
    hour = 'HOUR'
    minute = 'MINUTE'
    second = 'SECOND'
    microsecond = 'MICROSECOND'


class SqlType:
    def __init__(self, name):
        self.name = name

    def __call__(self, length):
        return SqlTypeLength(self.name, length)

    def get_sql(self, **kwargs):
        return '{name}'.format(name=self.name)


class SqlTypeLength:
    def __init__(self, name, length):
        self.name = name
        self.length = length

    def get_sql(self, **kwargs):
        return '{name}({length})'.format(name=self.name,
                                         length=self.length)


class SqlTypes:
    BOOLEAN = 'BOOLEAN'
    INTEGER = 'INTEGER'
    FLOAT = 'FLOAT'
    NUMERIC = 'NUMERIC'
    SIGNED = 'SIGNED'
    UNSIGNED = 'UNSIGNED'

    DATE = 'DATE'
    TIME = 'TIME'
    TIMESTAMP = 'TIMESTAMP'

    CHAR = SqlType('CHAR')
    VARCHAR = SqlType('VARCHAR')
    LONG_VARCHAR = SqlType('LONG VARCHAR')
    BINARY = SqlType('BINARY')
    VARBINARY = SqlType('VARBINARY')
    LONG_VARBINARY = SqlType('LONG VARBINARY')


class Dialects(Enum):
    VERTICA = 'vertica'
    CLICKHOUSE = 'clickhouse'
    ORACLE = 'oracle'
    MSSQL = 'mssql'
    MYSQL = 'mysql'
    POSTGRESQL = 'postgressql'
    REDSHIFT = 'redshift'
    SQLLITE = 'sqllite'

In [ ]:
# coding: utf-8
"""
Package for SQL functions wrappers
"""
from pypika.enums import (
    SqlTypes,
)
from pypika.terms import (
    AggregateFunction,
    Function,
    Star,
)
from pypika.utils import builder

__author__ = "Timothy Heys"
__email__ = "theys@kayak.com"


class DistinctOptionFunction(AggregateFunction):
    def __init__(self, name, *args, **kwargs):
        alias = kwargs.get('alias')
        super(DistinctOptionFunction, self).__init__(name, *args, alias=alias)
        self._distinct = False

    def get_function_sql(self, **kwargs):
        s = super(DistinctOptionFunction, self).get_function_sql(**kwargs)

        n = len(self.name) + 1
        if self._distinct:
            return s[:n] + 'DISTINCT ' + s[n:]
        return s

    @builder
    def distinct(self):
        self._distinct = True


class Count(DistinctOptionFunction):
    def __init__(self, param, alias=None):
        is_star = isinstance(param, str) and '*' == param
        super(Count, self).__init__('COUNT', Star() if is_star else param, alias=alias)


# Arithmetic Functions
class Sum(DistinctOptionFunction):
    def __init__(self, term, alias=None):
        super(Sum, self).__init__('SUM', term, alias=alias)


class Avg(AggregateFunction):
    def __init__(self, term, alias=None):
        super(Avg, self).__init__('AVG', term, alias=alias)


class Min(AggregateFunction):
    def __init__(self, term, alias=None):
        super(Min, self).__init__('MIN', term, alias=alias)


class Max(AggregateFunction):
    def __init__(self, term, alias=None):
        super(Max, self).__init__('MAX', term, alias=alias)


class Std(AggregateFunction):
    def __init__(self, term, alias=None):
        super(Std, self).__init__('STD', term, alias=alias)


class StdDev(AggregateFunction):
    def __init__(self, term, alias=None):
        super(StdDev, self).__init__('STDDEV', term, alias=alias)


class Abs(AggregateFunction):
    def __init__(self, term, alias=None):
        super(Abs, self).__init__('ABS', term, alias=alias)


class Sqrt(Function):
    def __init__(self, term, alias=None):
        super(Sqrt, self).__init__('SQRT', term, alias=alias)


class Floor(Function):
    def __init__(self, term, alias=None):
        super(Floor, self).__init__('FLOOR', term, alias=alias)


# Type Functions
class Cast(Function):
    def __init__(self, term, as_type, alias=None):
        super(Cast, self).__init__('CAST', term, alias=alias)
        self.as_type = as_type

    def get_special_params_sql(self, **kwargs):
        type_sql = self.as_type.get_sql(**kwargs) \
            if hasattr(self.as_type, 'get_sql') \
            else str(self.as_type).upper()

        return 'AS {type}'.format(type=type_sql)


class Convert(Function):
    def __init__(self, term, encoding, alias=None):
        super(Convert, self).__init__('CONVERT', term, alias=alias)
        self.encoding = encoding

    def get_special_params_sql(self, **kwargs):
        return 'USING {type}'.format(type=self.encoding.value)


class ToChar(Function):
    def __init__(self, term, as_type, alias=None):
        super(ToChar, self).__init__('TO_CHAR', term, as_type, alias=alias)


class Signed(Cast):
    def __init__(self, term, alias=None):
        super(Signed, self).__init__(term, SqlTypes.SIGNED, alias=alias)


class Unsigned(Cast):
    def __init__(self, term, alias=None):
        super(Unsigned, self).__init__(term, SqlTypes.UNSIGNED, alias=alias)


class Date(Function):
    def __init__(self, term, alias=None):
        super(Date, self).__init__('DATE', term, alias=alias)


class DateDiff(Function):
    def __init__(self, interval, start_date, end_date, alias=None):
        super(DateDiff, self).__init__('DATEDIFF', interval, start_date, end_date, alias=alias)


class DateAdd(Function):
    def __init__(self, date_part, interval, term, alias=None):
        super(DateAdd, self).__init__('DATE_ADD', date_part, interval, term, alias=alias)


class Timestamp(Function):
    def __init__(self, term, alias=None):
        super(Timestamp, self).__init__('TIMESTAMP', term, alias=alias)


class TimestampAdd(Function):
    def __init__(self, date_part, interval, term, alias=None):
        super(TimestampAdd, self).__init__('TIMESTAMPADD', date_part, interval, term, alias=alias)


# String Functions
class Ascii(Function):
    def __init__(self, term, alias=None):
        super(Ascii, self).__init__('ASCII', term, alias=alias)


class NullIf(Function):
    def __init__(self, criterion, alias=None):
        super(NullIf, self).__init__('NULLIF', criterion, alias=alias)


class Bin(Function):
    def __init__(self, term, alias=None):
        super(Bin, self).__init__('BIN', term, alias=alias)


class Concat(Function):
    def __init__(self, *terms, **kwargs):
        super(Concat, self).__init__('CONCAT', *terms, **kwargs)


class Insert(Function):
    def __init__(self, term, start, stop, subterm, alias=None):
        term, start, stop, subterm = [term for term in [term, start, stop, subterm]]
        super(Insert, self).__init__('INSERT', term, start, stop, subterm, alias=alias)


class Length(Function):
    def __init__(self, term, alias=None):
        super(Length, self).__init__('LENGTH', term, alias=alias)


class Upper(Function):
    def __init__(self, term, alias=None):
        super(Upper, self).__init__('UPPER', term, alias=alias)


class Lower(Function):
    def __init__(self, term, alias=None):
        super(Lower, self).__init__('LOWER', term, alias=alias)


class Substring(Function):
    def __init__(self, term, start, stop, alias=None):
        super(Substring, self).__init__('SUBSTRING', term, start, stop, alias=alias)


class Reverse(Function):
    def __init__(self, term, alias=None):
        super(Reverse, self).__init__('REVERSE', term, alias=alias)


class Trim(Function):
    def __init__(self, term, alias=None):
        super(Trim, self).__init__('TRIM', term, alias=alias)


class SplitPart(Function):
    def __init__(self, term, delimiter, index, alias=None):
        super(SplitPart, self).__init__('SPLIT_PART', term, delimiter, index, alias=alias)


class RegexpMatches(Function):
    def __init__(self, term, pattern, modifiers=None, alias=None):
        super(RegexpMatches, self).__init__('REGEXP_MATCHES', term, pattern, modifiers, alias=alias)


class RegexpLike(Function):
    def __init__(self, term, pattern, modifiers=None, alias=None):
        super(RegexpLike, self).__init__('REGEXP_LIKE', term, pattern, modifiers, alias=alias)


# Date/Time Functions
class Now(Function):
    def __init__(self, alias=None):
        super(Now, self).__init__('NOW', alias=alias)


class UtcTimestamp(Function):
    def __init__(self, alias=None):
        super(UtcTimestamp, self).__init__('UTC_TIMESTAMP', alias=alias)


class CurDate(Function):
    def __init__(self, alias=None):
        super(CurDate, self).__init__('CURRENT_DATE', alias=alias)


class CurTime(Function):
    def __init__(self, alias=None):
        super(CurTime, self).__init__('CURRENT_TIME', alias=alias)


class Extract(Function):
    def __init__(self, date_part, field, alias=None):
        super(Extract, self).__init__('EXTRACT', date_part, alias=alias)
        self.field = field

    def get_special_params_sql(self, **kwargs):
        return 'FROM {field}'.format(
            field=self.field,
        )


# Null Functions
class IsNull(Function):
    def __init__(self, term, alias=None):
        super(IsNull, self).__init__('ISNULL', term, alias=alias)


class Coalesce(Function):
    def __init__(self, term, *default_values, **kwargs):
        super(Coalesce, self).__init__('COALESCE', term, *default_values, **kwargs)


class IfNull(Function):
    def __init__(self, condition, term, **kwargs):
        super(IfNull, self).__init__('IFNULL', condition, term, **kwargs)


class NVL(Function):
    def __init__(self, condition, term, alias=None):
        super(NVL, self).__init__('NVL', condition, term, alias=alias)

In [ ]:
from .terms import PseudoColumn

ColumnValue = PseudoColumn('COLUMN_VALUE')
ObjectID = PseudoColumn('OBJECT_ID')
ObjectValue = PseudoColumn('OBJECT_VALUE')
RowNum = PseudoColumn('ROWNUM')
RowID = PseudoColumn('ROWID')
SysDate = PseudoColumn('SYSDATE')
© 2019 GitHub, Inc.

In [ ]:
import inspect
import itertools
import re
from datetime import date

from aenum import Enum

from pypika.enums import (
    Arithmetic,
    Boolean,
    Dialects,
    Equality,
    Matching,
)
from pypika.utils import (
    CaseException,
    alias_sql,
    builder,
    ignore_copy,
    resolve_is_aggregate,
)

try:
    basestring
except NameError:
    basestring = str

__author__ = "Timothy Heys"
__email__ = "theys@kayak.com"


class Term(object):
    is_aggregate = False

    def __init__(self, alias=None):
        self.alias = alias

    @builder
    def as_(self, alias):
        self.alias = alias

    @property
    def tables_(self):
        return set()

    def wrap_constant(self, val):
        """
        Used for wrapping raw inputs such as numbers in Criterions and Operator.
        For example, the expression F('abc')+1 stores the integer part in a ValueWrapper object.
        :param val:
            Any value.
        :return:
            Raw string, number, or decimal values will be returned in a ValueWrapper.  Fields and other parts of the
            querybuilder will be returned as inputted.
        """
        from .queries import QueryBuilder

        if isinstance(val, (Term, QueryBuilder, Interval)):
            return val
        if val is None:
            return NullValue()
        if isinstance(val, list):
            return Array(*val)
        if isinstance(val, tuple):
            return Tuple(*val)

        _ValueWrapper = getattr(self, '_wrapper_cls', ValueWrapper)
        return _ValueWrapper(val)

    def for_(self, table):
        """
        Replaces the tables of this term for the table parameter provided.  The base implementation returns self
        because not all terms have a table property.
        :param table:
            The table to replace with.
        :return:
            Self.
        """
        return self

    def fields(self):
        return [self]

    def eq(self, other):
        return self == other

    def isnull(self):
        return NullCriterion(self)

    def notnull(self):
        return self.isnull().negate()

    def gt(self, other):
        return self > other

    def gte(self, other):
        return self >= other

    def lt(self, other):
        return self < other

    def lte(self, other):
        return self <= other

    def ne(self, other):
        return self != other

    def like(self, expr):
        return BasicCriterion(Matching.like, self, self.wrap_constant(expr))

    def not_like(self, expr):
        return BasicCriterion(Matching.not_like, self, self.wrap_constant(expr))

    def ilike(self, expr):
        return BasicCriterion(Matching.ilike, self, self.wrap_constant(expr))

    def not_ilike(self, expr):
        return BasicCriterion(Matching.not_ilike, self, self.wrap_constant(expr))

    def regex(self, pattern):
        return BasicCriterion(Matching.regex, self, self.wrap_constant(pattern))

    def between(self, lower, upper):
        return BetweenCriterion(self, self.wrap_constant(lower), self.wrap_constant(upper))

    def isin(self, arg):
        if isinstance(arg, (list, tuple, set)):
            return ContainsCriterion(self, Tuple(*[self.wrap_constant(value) for value in arg]))
        return ContainsCriterion(self, arg)

    def notin(self, arg):
        return self.isin(arg).negate()

    def bin_regex(self, pattern):
        return BasicCriterion(Matching.bin_regex, self, self.wrap_constant(pattern))

    def negate(self):
        return Not(self)

    def __invert__(self):
        return Not(self)

    def __pos__(self):
        return self

    def __neg__(self):
        return Negative(self)

    def __add__(self, other):
        return ArithmeticExpression(Arithmetic.add, self, self.wrap_constant(other))

    def __sub__(self, other):
        return ArithmeticExpression(Arithmetic.sub, self, self.wrap_constant(other))

    def __mul__(self, other):
        return ArithmeticExpression(Arithmetic.mul, self, self.wrap_constant(other))

    def __div__(self, other):
        # Required for Python2
        return self.__truediv__(other)

    def __truediv__(self, other):
        return ArithmeticExpression(Arithmetic.div, self, self.wrap_constant(other))

    def __pow__(self, other):
        return Pow(self, other)

    def __mod__(self, other):
        return Mod(self, other)

    def __radd__(self, other):
        return ArithmeticExpression(Arithmetic.add, self.wrap_constant(other), self)

    def __rsub__(self, other):
        return ArithmeticExpression(Arithmetic.sub, self.wrap_constant(other), self)

    def __rmul__(self, other):
        return ArithmeticExpression(Arithmetic.mul, self.wrap_constant(other), self)

    def __rdiv__(self, other):
        # Required for Python2
        return self.__rtruediv__(other)

    def __rtruediv__(self, other):
        return ArithmeticExpression(Arithmetic.div, self.wrap_constant(other), self)

    def __eq__(self, other):
        return BasicCriterion(Equality.eq, self, self.wrap_constant(other))

    def __ne__(self, other):
        return BasicCriterion(Equality.ne, self, self.wrap_constant(other))

    def __gt__(self, other):
        return BasicCriterion(Equality.gt, self, self.wrap_constant(other))

    def __ge__(self, other):
        return BasicCriterion(Equality.gte, self, self.wrap_constant(other))

    def __lt__(self, other):
        return BasicCriterion(Equality.lt, self, self.wrap_constant(other))

    def __le__(self, other):
        return BasicCriterion(Equality.lte, self, self.wrap_constant(other))

    def __getitem__(self, item):
        if not isinstance(item, slice):
            raise TypeError("Field' object is not subscriptable")
        return self.between(item.start, item.stop)

    def __str__(self):
        return self.get_sql(quote_char='"')

    def __hash__(self):
        return hash(self.get_sql(with_alias=True))

    def get_sql(self):
        raise NotImplementedError()


class Parameter(Term):
    is_aggregate = None

    def __init__(self, placeholder):
        super(Parameter, self).__init__()
        self.placeholder = placeholder

    def fields(self):
        return []

    def get_sql(self, **kwargs):
        return str(self.placeholder)


class Negative(Term):
    def __init__(self, term):
        super(Negative, self).__init__()
        self.term = term

    def get_sql(self, **kwargs):
        return '-{term}'.format(term=self.term.get_sql(**kwargs))


class ValueWrapper(Term):
    is_aggregate = None

    def __init__(self, value, alias=None):
        super(ValueWrapper, self).__init__(alias)
        self.value = value

    def fields(self):
        return []

    def get_value_sql(self, quote_char=None, **kwargs):
        # FIXME escape values
        if isinstance(self.value, Term):
            return self.value.get_sql(quote_char=quote_char, **kwargs)
        if isinstance(self.value, Enum):
            return self.value.value
        if isinstance(self.value, date):
            return "'%s'" % self.value.isoformat()
        if isinstance(self.value, basestring):
            value = self.value.replace("'", "''")
            return "'%s'" % value
        if isinstance(self.value, bool):
            return str.lower(str(self.value))
        if self.value is None:
            return 'null'
        return str(self.value)

    def get_sql(self, quote_char=None, **kwargs):
        sql = self.get_value_sql(quote_char=quote_char, **kwargs)
        return alias_sql(sql, self.alias, quote_char)


class Values(Term):
    def __init__(self, field, ):
        super(Values, self).__init__(None)
        self.field = Field(field) if not isinstance(field, Field) else field

    def get_sql(self, quote_char=None, **kwargs):
        return 'VALUES({value})'.format(value=self.field.get_sql(quote_char=quote_char, **kwargs))


class NullValue(Term):
    def fields(self):
        return []

    def get_sql(self, quote_char=None, **kwargs):
        sql = 'NULL'
        return alias_sql(sql, self.alias, quote_char)


class Criterion(Term):
    def __and__(self, other):
        return ComplexCriterion(Boolean.and_, self, other)

    def __or__(self, other):
        return ComplexCriterion(Boolean.or_, self, other)

    def __xor__(self, other):
        return ComplexCriterion(Boolean.xor_, self, other)

    @staticmethod
    def any(terms=()):
        crit = EmptyCriterion()

        for term in terms:
            crit |= term

        return crit

    @staticmethod
    def all(terms=()):
        crit = EmptyCriterion()

        for term in terms:
            crit &= term

        return crit

    def fields(self):
        raise NotImplementedError()

    def get_sql(self):
        raise NotImplementedError()


class EmptyCriterion:
    def __and__(self, other):
        return other

    def __or__(self, other):
        return other

    def __xor__(self, other):
        return other


class Field(Criterion):
    def __init__(self, name, alias=None, table=None):
        super(Field, self).__init__(alias)
        self.name = name
        self.table = table

    def fields(self):
        return [self]

    @property
    def tables_(self):
        return {self.table}

    @builder
    def for_(self, table):
        """
        Replaces the tables of this term for the table parameter provided.  Useful when reusing fields across queries.
        :param table:
            The table to replace with.
        :return:
            A copy of the field with it's table value replaced.
        """
        self.table = table

    def get_sql(self, with_alias=False, with_namespace=False, quote_char=None, **kwargs):
        # Need to add namespace if the table has an alias
        if self.table and (with_namespace or self.table.alias):
            field_sql = "{quote}{namespace}{quote}.{quote}{name}{quote}".format(
                namespace=self.table.alias or self.table._table_name,
                name=self.name,
                quote=quote_char or '',
            )
        else:
            field_sql = "{quote}{name}{quote}".format(
                name=self.name,
                quote=quote_char or '',
            )

        field_alias = getattr(self, 'alias', None)
        if not with_alias or field_alias is None:
            return field_sql

        return alias_sql(field_sql, field_alias, quote_char)


class Star(Field):
    def __init__(self, table=None):
        super(Star, self).__init__('*', table=table)

    @property
    def tables_(self):
        if self.table is None:
            return {}
        return {self.table}

    def get_sql(self, with_alias=False, with_namespace=False, quote_char=None, **kwargs):
        if self.table and (with_namespace or self.table.alias):
            return "{quote}{namespace}{quote}.*".format(
                namespace=self.table.alias or getattr(self.table, '_table_name'),
                quote=quote_char or ''
            )

        return '*'


class Tuple(Criterion):
    def __init__(self, *values):
        super(Tuple, self).__init__()
        self.values = [self.wrap_constant(value)
                       for value in values]

    def fields(self):
        return list(itertools.chain(*[value.fields()
                                      for value in self.values]))

    def get_sql(self, **kwargs):
        return '({})'.format(
            ','.join(term.get_sql(**kwargs)
                     for term in self.values)
        )

    @property
    def is_aggregate(self):
        return all([value.is_aggregate
                    for value in self.values])


class Array(Tuple):
    def get_sql(self, **kwargs):
        return '[{}]'.format(
            ','.join(term.get_sql(**kwargs)
                     for term in self.values)
        )


class Bracket(Tuple):
    def __init__(self, term):
        super(Bracket, self).__init__(term)

    def get_sql(self, **kwargs):
        return alias_sql(
            sql=super(Bracket, self).get_sql(**kwargs),
            alias=self.alias,
            quote_char=kwargs.get('quote_char', None),
        )


class BasicCriterion(Criterion):
    def __init__(self, comparator, left, right, alias=None):
        """
        A wrapper for a basic criterion such as equality or inequality. This wraps three parts, a left and right term
        and a comparator which defines the type of comparison.
        :param comparator:
            Type: Comparator
            This defines the type of comparison, such as {quote}={quote} or {quote}>{quote}.
        :param left:
            The term on the left side of the expression.
        :param right:
            The term on the right side of the expression.
        """
        super(BasicCriterion, self).__init__(alias)
        self.comparator = comparator
        self.left = left
        self.right = right

    @property
    def is_aggregate(self):
        return resolve_is_aggregate([term.is_aggregate for term in [self.left, self.right]])

    @property
    def tables_(self):
        return self.left.tables_ | self.right.tables_

    @builder
    def for_(self, table):
        self.left = self.left.for_(table)
        self.right = self.right.for_(table)

    def fields(self):
        return self.left.fields() + self.right.fields()

    def get_sql(self, with_alias=False, **kwargs):
        sql = '{left}{comparator}{right}'.format(
            comparator=self.comparator.value,
            left=self.left.get_sql(**kwargs),
            right=self.right.get_sql(**kwargs),
        )
        if with_alias and self.alias:
            return '{sql} "{alias}"'.format(sql=sql, alias=self.alias)

        return sql


class ContainsCriterion(Criterion):
    def __init__(self, term, container, alias=None):
        """
        A wrapper for a "IN" criterion.  This wraps two parts, a term and a container.  The term is the part of the
        expression that is checked for membership in the container.  The container can either be a list or a subquery.
        :param term:
            The term to assert membership for within the container.
        :param container:
            A list or subquery.
        """
        super(ContainsCriterion, self).__init__(alias)
        self.term = term
        self.container = container
        self._is_negated = False

    @property
    def tables_(self):
        return self.term.tables_

    @property
    def is_aggregate(self):
        return self.term.is_aggregate

    def fields(self):
        return self.term.fields() if self.term.fields else []

    def get_sql(self, **kwargs):
        # FIXME escape
        return "{term} {not_}IN {container}".format(
            term=self.term.get_sql(**kwargs),
            container=self.container.get_sql(**kwargs),
            not_='NOT ' if self._is_negated else ''
        )

    def negate(self):
        self._is_negated = True
        return self


class BetweenCriterion(Criterion):
    def __init__(self, term, start, end, alias=None):
        super(BetweenCriterion, self).__init__(alias)
        self.term = term
        self.start = start
        self.end = end

    @property
    def tables_(self):
        return self.term.tables_

    @property
    def is_aggregate(self):
        return self.term.is_aggregate

    @builder
    def for_(self, table):
        self.term = self.term.for_(table)

    def get_sql(self, **kwargs):
        # FIXME escape
        return "{term} BETWEEN {start} AND {end}".format(
            term=self.term.get_sql(**kwargs),
            start=self.start.get_sql(**kwargs),
            end=self.end.get_sql(**kwargs),
        )

    def fields(self):
        return self.term.fields() if self.term.fields else []


class NullCriterion(Criterion):
    def __init__(self, term, alias=None):
        super(NullCriterion, self).__init__(alias)
        self.term = term

    @property
    def tables_(self):
        return self.term.tables_

    @builder
    def for_(self, table):
        self.term = self.term.for_(table)

    def get_sql(self, **kwargs):
        return "{term} IS NULL".format(
            term=self.term.get_sql(**kwargs),
        )

    def fields(self):
        return self.term.fields() if self.term.fields else []


class ComplexCriterion(BasicCriterion):
    def fields(self):
        return self.left.fields() + self.right.fields()

    def get_sql(self, subcriterion=False, **kwargs):
        sql = '{left} {comparator} {right}'.format(
            comparator=self.comparator.value,
            left=self.left.get_sql(subcriterion=self.needs_brackets(self.left), **kwargs),
            right=self.right.get_sql(subcriterion=self.needs_brackets(self.right), **kwargs),
        )

        if subcriterion:
            return '({criterion})'.format(
                criterion=sql
            )

        return sql

    def needs_brackets(self, term):
        return isinstance(term, ComplexCriterion) and not term.comparator == self.comparator


class ArithmeticExpression(Term):
    """
    Wrapper for an arithmetic function.  Can be simple with two terms or complex with nested terms. Order of operations
    are also preserved.
    """

    mul_order = [Arithmetic.mul, Arithmetic.div]
    add_order = [Arithmetic.add, Arithmetic.sub]

    def __init__(self, operator, left, right, alias=None):
        """
        Wrapper for an arithmetic expression.
        :param operator:
            Type: Arithmetic
            An operator for the expression such as {quote}+{quote} or {quote}/{quote}
        :param left:
            The term on the left side of the expression.
        :param right:
            The term on the right side of the expression.
        :param alias:
            (Optional) an alias for the term which can be used inside a select statement.
        :return:
        """
        super(ArithmeticExpression, self).__init__(alias)
        self.operator = operator
        self.left = left
        self.right = right

    @property
    def is_aggregate(self):
        # True if both left and right terms are True or None. None if both terms are None. Otherwise, False
        return resolve_is_aggregate([self.left.is_aggregate, self.right.is_aggregate])

    @property
    def tables_(self):
        return self.left.tables_ | self.right.tables_

    @builder
    def for_(self, table):
        """
        Replaces the tables of this term for the table parameter provided.  Useful when reusing terms across queries.
        :param table:
            The table to replace with.
        :return:
            A copy of the term with it's table value replaced.
        """
        self.left = self.left.for_(table)
        self.right = self.right.for_(table)

    def fields(self):
        return self.left.fields() + self.right.fields()

    def get_sql(self, with_alias=False, **kwargs):
        is_mul = self.operator in self.mul_order
        is_left_add, is_right_add = [getattr(side, 'operator', None) in self.add_order
                                     for side in [self.left, self.right]]

        quote_char = kwargs.get('quote_char', None)
        arithmatic_sql = '{left}{operator}{right}'.format(
            operator=self.operator.value,
            left=("({})" if is_mul and is_left_add else "{}").format(self.left.get_sql(**kwargs)),
            right=("({})" if is_mul and is_right_add else "{}").format(self.right.get_sql(**kwargs)),
        )

        if not with_alias or self.alias is None:
            return arithmatic_sql

        return alias_sql(arithmatic_sql, self.alias, quote_char)


class Case(Term):
    def __init__(self, alias=None):
        super(Case, self).__init__(alias=alias)
        self._cases = []
        self._else = None

    @property
    def is_aggregate(self):
        # True if all cases are True or None. None all cases are None. Otherwise, False
        return resolve_is_aggregate([term.is_aggregate for _, term in self._cases]
                                    + [self._else.is_aggregate if self._else else None])

    @builder
    def when(self, criterion, term):
        self._cases.append((criterion, self.wrap_constant(term)))

    @builder
    def else_(self, term):
        self._else = self.wrap_constant(term)
        return self

    def get_sql(self, with_alias=False, **kwargs):
        if not self._cases:
            raise CaseException("At least one 'when' case is required for a CASE statement.")

        cases = " ".join('WHEN {when} THEN {then}'.format(
            when=criterion.get_sql(**kwargs),
            then=term.get_sql(**kwargs)
        ) for criterion, term in self._cases)
        else_ = (' ELSE {}'.format(self._else.get_sql(**kwargs))
                 if self._else
                 else '')

        case_sql = 'CASE {cases}{else_} END'.format(cases=cases, else_=else_)

        if not with_alias or self.alias is None:
            return case_sql

        return alias_sql(case_sql, self.alias, kwargs.get('quote_char'))

    def fields(self):
        fields = []

        for criterion, term in self._cases:
            fields += criterion.fields() + term.fields()

        if self._else is not None:
            fields += self._else.fields()

        return fields

    @property
    def tables_(self):
        tables = set()
        if self._cases:
            tables |= {table
                       for case in self._cases
                       for part in case
                       for table in part.tables_
                       if hasattr(part, 'tables_')}

        if self._else and hasattr(self._else, 'tables_'):
            tables |= {table
                       for table in self._else.tables_}

        return tables


class Not(Criterion):
    def __init__(self, term, alias=None):
        super(Not, self).__init__(alias=alias)
        self.term = term

    def fields(self):
        return self.term.fields() if self.term.fields else []

    def get_sql(self, quote_char=None, **kwargs):
        kwargs['subcriterion'] = True
        sql = "NOT {term}".format(term=self.term.get_sql(quote_char=quote_char,
                                                         **kwargs))
        return alias_sql(sql, self.alias, quote_char=quote_char)

    @ignore_copy
    def __getattr__(self, name):
        """
        Delegate method calls to the class wrapped by Not().
        Re-wrap methods on child classes of Term (e.g. isin, eg...) to retain 'NOT <term>' output.
        """
        item_func = getattr(self.term, name)

        if not inspect.ismethod(item_func):
            return item_func

        def inner(inner_self, *args, **kwargs):
            result = item_func(inner_self, *args, **kwargs)
            if isinstance(result, (Term,)):
                return Not(result)
            return result

        return inner

    @property
    def tables_(self):
        return self.term.tables_


class Function(Criterion):
    def __init__(self, name, *args, **kwargs):
        super(Function, self).__init__(kwargs.get('alias'))
        self.name = name
        self.args = [self.wrap_constant(param)
                     for param in args]
        self.schema = kwargs.get('schema')

    @property
    def tables_(self):
        return {table
                for param in self.args
                for table in param.tables_}

    def fields(self):
        return [field
                for param in self.args
                if hasattr(param, 'fields')
                for field in param.fields()]

    @property
    def is_aggregate(self):
        """
        This is a shortcut thst assumes if a function has a single argument and that argument is aggregated, then this
        function is also aggregated. A more sophisticated approach is needed, however it is unclear how that might work.
        :returns:
            True if the function accepts one argument and that argument is aggregate.
        """
        return len(self.args) == 1 and self.args[0].is_aggregate

    @builder
    def for_(self, table):
        """
        Replaces the tables of this term for the table parameter provided.  Useful when reusing fields across queries.
        :param table:
            The table to replace with.
        :return:
            A copy of the field with it's table value replaced.
        """
        self.args = [param.for_(table) for param in self.args]

    def get_special_params_sql(self, **kwargs):
        pass

    def get_function_sql(self, **kwargs):
        special_params_sql = self.get_special_params_sql(**kwargs)

        return '{name}({args}{special})'.format(
            name=self.name,
            args=','.join(p.get_sql(with_alias=False, **kwargs)
                          if hasattr(p, 'get_sql')
                          else str(p)
                          for p in self.args),
            special=(' ' + special_params_sql) if special_params_sql else '',
        )

    def get_sql(self, with_alias=False, with_namespace=False, quote_char=None, **kwargs):
        # FIXME escape
        function_sql = self.get_function_sql(with_namespace=with_namespace, quote_char=quote_char)

        if self.schema is not None:
            function_sql = '{schema}.{function}' \
                .format(schema=self.schema.get_sql(quote_char=quote_char, **kwargs),
                        function=function_sql)

        if not with_alias or self.alias is None:
            return function_sql

        return alias_sql(function_sql, self.alias, quote_char)


class AggregateFunction(Function):
    is_aggregate = True


class AnalyticFunction(Function):
    is_analytic = True

    def __init__(self, name, *args, **kwargs):
        super(AnalyticFunction, self).__init__(name, *args, **kwargs)
        self._partition = []
        self._orderbys = []

    @builder
    def over(self, *terms):
        self._partition += terms

    @builder
    def orderby(self, *terms, **kwargs):
        self._orderbys += [(term, kwargs.get('order'))
                           for term in terms]

    def _orderby_field(self, field, orient, **kwargs):
        if orient is None:
            return field.get_sql(**kwargs)

        return '{field} {orient}'.format(
            field=field.get_sql(**kwargs),
            orient=orient.value,
        )

    def get_partition_sql(self, **kwargs):
        terms = []
        if self._partition:
            terms.append('PARTITION BY {args}'.format(
                args=','.join(p.get_sql(**kwargs)
                              if hasattr(p, 'get_sql')
                              else str(p)
                              for p in self._partition)))

        if self._orderbys:
            terms.append('ORDER BY {orderby}'.format(
                orderby=','.join(
                    self._orderby_field(field, orient, **kwargs)
                    for field, orient in self._orderbys
                )))

        return ' '.join(terms)

    def get_function_sql(self, **kwargs):
        function_sql = super(AnalyticFunction, self).get_function_sql(**kwargs)
        partition_sql = self.get_partition_sql(**kwargs)

        if not partition_sql:
            return function_sql

        return '{function_sql} OVER({partition_sql})'.format(
            function_sql=function_sql,
            partition_sql=partition_sql
        )


class WindowFrameAnalyticFunction(AnalyticFunction):
    class Edge:
        def __init__(self, value=None):
            self.value = value

        def __str__(self):
            return '{value} {modifier}'.format(
                value=self.value or 'UNBOUNDED',
                modifier=self.modifier,
            )

    def __init__(self, name, *args, **kwargs):
        super(WindowFrameAnalyticFunction, self).__init__(name, *args, **kwargs)
        self.frame = None
        self.bound = None

    def _set_frame_and_bounds(self, frame, bound, and_bound):
        if self.frame or self.bound:
            raise AttributeError()

        self.frame = frame
        self.bound = (bound, and_bound) if and_bound else bound

    @builder
    def rows(self, bound, and_bound=None):
        self._set_frame_and_bounds('ROWS', bound, and_bound)

    @builder
    def range(self, bound, and_bound=None):
        self._set_frame_and_bounds('RANGE', bound, and_bound)

    def get_frame_sql(self):
        if not isinstance(self.bound, tuple):
            return '{frame} {bound}'.format(
                frame=self.frame,
                bound=self.bound
            )

        lower, upper = self.bound
        return '{frame} BETWEEN {lower} AND {upper}'.format(
            frame=self.frame,
            lower=lower,
            upper=upper,
        )

    def get_partition_sql(self, **kwargs):
        partition_sql = super(WindowFrameAnalyticFunction, self).get_partition_sql(**kwargs)

        if not self.frame and not self.bound:
            return partition_sql

        return '{over} {frame}'.format(
            over=partition_sql,
            frame=self.get_frame_sql()
        )


class IgnoreNullsAnalyticFunction(AnalyticFunction):
    def __init__(self, name, *args, **kwargs):
        super(IgnoreNullsAnalyticFunction, self).__init__(name, *args, **kwargs)
        self._ignore_nulls = False

    @builder
    def ignore_nulls(self):
        self._ignore_nulls = True

    def get_special_params_sql(self, **kwargs):
        if self._ignore_nulls:
            return 'IGNORE NULLS'

        # No special params unless ignoring nulls
        return None


class Interval(object):
    templates = {
        # MySQL requires no single quotes around the expr and unit
        Dialects.MYSQL: 'INTERVAL {expr} {unit}',

        # PostgreSQL, Redshift and Vertica require quotes around the expr and unit e.g. INTERVAL '1 week'
        Dialects.POSTGRESQL: 'INTERVAL \'{expr} {unit}\'',
        Dialects.REDSHIFT: 'INTERVAL \'{expr} {unit}\'',
        Dialects.VERTICA: 'INTERVAL \'{expr} {unit}\'',

        # Oracle requires just single quotes around the expr
        Dialects.ORACLE: 'INTERVAL \'{expr}\' {unit}'
    }

    units = ['years', 'months', 'days', 'hours', 'minutes', 'seconds', 'microseconds']
    labels = ['YEAR', 'MONTH', 'DAY', 'HOUR', 'MINUTE', 'SECOND', 'MICROSECOND']

    trim_pattern = re.compile(r'(^0+\.)|(\.0+$)|(^[0\-.: ]+[\-: ])|([\-:. ][0\-.: ]+$)')

    def __init__(self, years=0, months=0, days=0, hours=0, minutes=0, seconds=0, microseconds=0, quarters=0, weeks=0,
                 dialect=None):
        self.dialect = dialect
        self.largest = None
        self.smallest = None

        if quarters:
            self.quarters = quarters
            return

        if weeks:
            self.weeks = weeks
            return

        for unit, label, value in zip(self.units, self.labels, [years, months, days,
                                                                hours, minutes, seconds, microseconds]):
            if value:
                setattr(self, unit, int(value))
                self.largest = self.largest or label
                self.smallest = label

    def __str__(self):
        return self.get_sql()

    def fields(self):
        return []

    def get_sql(self, **kwargs):
        dialect = self.dialect or kwargs.get('dialect')

        if self.largest == 'MICROSECOND':
            expr = getattr(self, 'microseconds')
            unit = 'MICROSECOND'

        elif hasattr(self, 'quarters'):
            expr = getattr(self, 'quarters')
            unit = 'QUARTER'

        elif hasattr(self, 'weeks'):
            expr = getattr(self, 'weeks')
            unit = 'WEEK'

        else:
            # Create the whole expression but trim out the unnecessary fields
            expr = "{years}-{months}-{days} {hours}:{minutes}:{seconds}.{microseconds}".format(
                years=getattr(self, 'years', 0),
                months=getattr(self, 'months', 0),
                days=getattr(self, 'days', 0),
                hours=getattr(self, 'hours', 0),
                minutes=getattr(self, 'minutes', 0),
                seconds=getattr(self, 'seconds', 0),
                microseconds=getattr(self, 'microseconds', 0),
            )
            expr = self.trim_pattern.sub('', expr)

            unit = '{largest}_{smallest}'.format(
                largest=self.largest,
                smallest=self.smallest,
            ) if self.largest != self.smallest else self.largest

        return self.templates.get(dialect, 'INTERVAL \'{expr} {unit}\'') \
            .format(expr=expr, unit=unit)


class Pow(Function):
    def __init__(self, term, exponent, alias=None):
        super(Pow, self).__init__('POW', term, exponent, alias=alias)


class Mod(Function):
    def __init__(self, term, modulus, alias=None):
        super(Mod, self).__init__('MOD', term, modulus, alias=alias)


class Rollup(Function):
    def __init__(self, *terms):
        super(Rollup, self).__init__('ROLLUP', *terms)


class PseudoColumn(Term):
    """
    Represents a pseudo column (a "column" which yields a value when selected
    but is not actually a real table column).
    """

    def __init__(self, name):
        self.name = name

    def get_sql(self, **kwargs):
        return self.name

    def fields(self):
        return []

In [ ]:
# coding: utf-8

__author__ = "Timothy Heys"
__email__ = "theys@kayak.com"


class QueryException(Exception):
    pass


class GroupingException(Exception):
    pass


class CaseException(Exception):
    pass


class JoinException(Exception):
    pass


class UnionException(Exception):
    pass


class RollupException(Exception):
    pass


class DialectNotSupported(Exception):
    pass


def builder(func):
    """
    Decorator for wrapper "builder" functions.  These are functions on the Query class or other classes used for
    building queries which mutate the query and return self.  To make the build functions immutable, this decorator is
    used which will deepcopy the current instance.  This decorator will return the return value of the inner function
    or the new copy of the instance.  The inner function does not need to return self.
    """
    import copy

    def _copy(self, *args, **kwargs):
        self_copy = copy.copy(self)
        result = func(self_copy, *args, **kwargs)

        # Return self if the inner function returns None.  This way the inner function can return something
        # different (for example when creating joins, a different builder is returned).
        if result is None:
            return self_copy

        return result

    return _copy


def ignore_copy(func):
    """
    Decorator for wrapping the __getattr__ function for classes that are copied via deepcopy.  This prevents infinite
    recursion caused by deepcopy looking for magic functions in the class. Any class implementing __getattr__ that is
    meant to be deepcopy'd should use this decorator.
    deepcopy is used by pypika in builder functions (decorated by @builder) to make the results immutable.  Any data
    model type class (stored in the Query instance) is copied.
    """

    def _getattr(self, name):
        if name in ['__copy__','__deepcopy__', '__getstate__', '__setstate__', '__getnewargs__']:
            raise AttributeError("'%s' object has no attribute '%s'" % (self.__class__.__name__, name))

        return func(self, name)

    return _getattr


def resolve_is_aggregate(values):
    """
    Resolves the is_aggregate flag for an expression that contains multiple terms.  This works like a voter system,
    each term votes True or False or abstains with None.
    :param values: A list of booleans (or None) for each term in the expression
    :return: If all values are True or None, True is returned.  If all values are None, None is returned. Otherwise,
        False is returned.
    """
    result = [x
              for x in values
              if x is not None]
    if result:
        return all(result)
    return None


def format_quotes(value, quote_char):
    return '{quote}{value}{quote}' \
        .format(value=value, quote=quote_char or '')


def alias_sql(sql, alias, quote_char=None):
    if alias is None:
        return sql
    return '{sql} {alias}'.format(sql=sql, alias=format_quotes(alias, quote_char))